In [1]:
import csv
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import random
import sys
from numpy.linalg import cholesky

# change it to your file directory
model_path ='/data/qit16/Lib/word2vec_twitter_model.bin'
source_file='/data/gesy/MNA/WebMD8000.csv'

In [2]:
# open file and get raw data
source=[]
drug=[]
easeofuse=[]
effective=[]
satisfy=[]
text=[]
adh=[]
disc=[]
ADR=[]
with open(source_file,encoding='ISO-8859-1') as file:
    csv_reader = csv.reader(file) 
    for row in csv_reader:  
        source.append(row[0].lower())
        drug.append(row[1])
        easeofuse.append(row[2])
        effective.append(row[3])
        satisfy.append(row[4])
        text.append(row[5])
        adh.append(row[6])
        disc.append(row[7])
        ADR.append(row[8])
source=source[1:]
drug=drug[1:]
easeofuse=easeofuse[1:]
effective=effective[1:]
satisfy=satisfy[1:]
text=text[1:]
adh=adh[1:]
disc=disc[1:]
ADR=ADR[1:]
labels=np.zeros(len(source))
for i in range(len(source)):
    if adh[i]=='1' or disc[i]=='1':
        labels[i]=1
easeofuse=np.array(easeofuse)
effective=np.array(effective)
satisfy=np.array(satisfy)
count=0
for d in drug:
    if d[0]=='user':
        d=d[5:]

In [3]:
def tokenize(texts):
    tokenized_texts=[]
    for text in texts:  
        from nltk.tokenize import TweetTokenizer
        tokenizer= TweetTokenizer(strip_handles=True, reduce_len = True, preserve_case=False)
        tokenized_text=tokenizer.tokenize(text)
        tokenized_texts.append(tokenized_text)
    return tokenized_texts

In [4]:
def generate_indices_embedding(tokenized_text,model_path):
    #change default directory to the one containing the module 'word2vecReader'
    sys.path.append('/data/qit16/Lib')
    from word2vecReader import Word2Vec
    
    #count apppearing times of words
    word_indices_count={'PADDING':[0,99999]}
    word_indices={'PADDING':0}
    embedding_index={'PADDING':0}
    for sentence in tokenized_text:
        for word in sentence:
            if word not in word_indices_count:
                word_indices_count[word]=[len(word_indices_count),1]
            else:
                word_indices_count[word][1]+=1
    for word in word_indices_count.keys():
        word_indices[word]=len(word_indices)
    total_dict=Word2Vec.load_word2vec_format(model_path, binary=True)
    print("The vocabulary size is: "+str(len(total_dict.vocab)))
    
    #generate new embeddings for out-of-vocabulary words
    count=0
    for word in word_indices.keys():
        if word in total_dict.vocab.keys():
            count+=1 
        elif word_indices_count[word][1]>1:
            count+=1
    lister=np.zeros((count,400),dtype='float32')
    
    for word in word_indices.keys():
        if word in total_dict.vocab.keys() and word!='PADDING':
            embedding_index[word]=len(embedding_index)
            lister[embedding_index[word]]=total_dict.syn0[total_dict.vocab[word].index]
    reference=lister[:len(embedding_index)-1]
    mu=np.mean(reference, axis=0)
    Sigma=np.cov(reference.T)
    for word in word_indices_count.keys():
        if word not in embedding_index.keys() and word_indices_count[word][1]>1:
            embedding_index[word]=len(embedding_index)
            lister[embedding_index[word]]=np.random.multivariate_normal(mu, Sigma, 1)
   
    # generate indices for tokenized text        
    text_word_indices=[]
    for text in tokenized_text:
        text_word_index=[]
        for word in text:
            if word in embedding_index:
                text_word_index.append(embedding_index[word])
            else:
                text_word_index.append(embedding_index['PADDING'])
        text_word_indices.append(text_word_index)
   
    return text_word_indices,lister

In [5]:
# tokenize the reviews
tokenized_text=tokenize(text)

In [6]:
# 'all_train' is the network input for tokenized text, 'lister' is the embedding matrix
all_train,lister=generate_indices_embedding(tokenized_text,model_path)

The vocabulary size is: 3039345


In [7]:
# define some modules in the network architecture
import tensorflow as tf
import sklearn
import keras
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers 
from keras.utils import plot_model
from sklearn.metrics import *
from keras.layers import *
from keras.optimizers import Optimizer
class Attention(Layer):
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12        
    def call(self, x):
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

class AttLayer(Layer):
    def __init__(self, **kwargs):
        # self.init = initializations.get('normal')#keras1.2.2
        self.init = initializers.get('normal')

        # self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], 1)))
        # self.W = self.init((input_shape[-1],))
        # self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        # eij = K.tanh(K.dot(x, self.W))
        #print(x.shape)
        #print(self.W.shape)
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        print(ai.shape)
        # weights = ai / K.sum(ai, axis=1).dimshuffle(0, 'x')
        weights = ai / K.expand_dims(K.sum(ai, axis=1), 1)
        #print('weights', weights.shape)
        # weighted_input = x * weights.dimshuffle(0, 1, 'x')
        weighted_input = x * weights

        # return weighted_input.sum(axis=1)
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

class Adam(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, **kwargs):
        super(Adam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates
myadam = Adam(lr=0.001,amsgrad=True)

Using TensorFlow backend.


In [8]:
# split data into training set and testing set
indices = np.arange(len(all_train))
np.random.shuffle(indices)
all_train=pad_sequences(all_train, maxlen=100, dtype='int32', padding='post', truncating='post', value=0)
train_sen=all_train[indices[:int(0.8*len(indices))]]
train_ease=easeofuse[indices[:int(0.8*len(indices))]]
train_effect=effective[indices[:int(0.8*len(indices))]]
train_satis=satisfy[indices[:int(0.8*len(indices))]]
train_labels=labels[indices[:int(0.8*len(indices))]]

val_sen=all_train[indices[int(0.8*len(indices)):]]
val_ease=easeofuse[indices[int(0.8*len(indices)):]]
val_effect=effective[indices[int(0.8*len(indices)):]]
val_satis=satisfy[indices[int(0.8*len(indices)):]]
val_labels=labels[indices[int(0.8*len(indices)):]]

In [9]:
# define the model architecture
text_input = Input(shape=(100,), dtype='int32')
ease_input= Input(shape=(1,), dtype='int32')
effect_input= Input(shape=(1,), dtype='int32')
satis_input=Input(shape=(1,), dtype='int32')

text_embedding_layer = Embedding(lister.shape[0],400,weights=[lister], trainable=True)
ease_embedding_layer = Embedding(5,5,input_length=1,trainable=True)
effect_embedding_layer = Embedding(5,5,input_length=1,trainable=True)
satis_embedding_layer = Embedding(2,5,input_length=1,trainable=True)

embedded_text = text_embedding_layer(text_input)
embedded_ease = ease_embedding_layer(ease_input)
embedded_effect = effect_embedding_layer(effect_input)
embedded_satis = satis_embedding_layer(satis_input)
d_embedded_text=Dropout(0.2)(embedded_text)
d_embedded_ease= Dropout(0.2)(embedded_ease)
d_embedded_effect= Dropout(0.2)(embedded_effect)
d_embedded_satis= Dropout(0.2)(embedded_satis)

cnn1=Conv1D(padding="same", activation="relu", strides=1, filters=300, kernel_size=3)(d_embedded_text)
d_cnn1= Dropout(0.2)(cnn1)
p=GlobalMaxPooling1D()(d_cnn1)
cp=concatenate([p,Flatten()(d_embedded_ease),Flatten()(d_embedded_effect),Flatten()(d_embedded_satis)],axis=-1)

dense= Dense(200, activation='relu')(cp)   
final= Dense(2, activation='softmax')(dense)
model = Model([text_input,ease_input,effect_input,satis_input], final)
model.compile(loss='categorical_crossentropy', optimizer=myadam, metrics=['acc'])

In [10]:
report_accuracy_score=[]
report=[]
class scoreHistory(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        predictions = model.predict([val_sen,val_ease,val_effect,val_satis], batch_size=32, verbose=1)
        predictions = np.argmax(predictions, axis=1)
        true = val_labels
        cr = classification_report(true, predictions,digits=4)
        acc_score=accuracy_score(true, predictions)
        print(acc_score)
        print(cr)
        report_accuracy_score.append(acc_score)
        report.append(cr)
scorehistory = scoreHistory()

#train the model and test every epoch using the callback function 'scoreHistory'
history=model.fit([train_sen,train_ease,train_effect,train_satis],to_categorical(train_labels),batch_size=64,callbacks=[scorehistory],
                   epochs=10,shuffle=True) 

Epoch 1/10
1738/1738 [==============================] - 2s 1ms/step
0.8043728423475259
              precision    recall  f1-score   support

         0.0     0.8323    0.8998    0.8648      1208
         1.0     0.7199    0.5868    0.6466       530

   micro avg     0.8044    0.8044    0.8044      1738
   macro avg     0.7761    0.7433    0.7557      1738
weighted avg     0.7980    0.8044    0.7982      1738

Epoch 2/10
1738/1738 [==============================] - 1s 586us/step
0.8227848101265823
              precision    recall  f1-score   support

         0.0     0.8623    0.8866    0.8743      1208
         1.0     0.7238    0.6774    0.6998       530

   micro avg     0.8228    0.8228    0.8228      1738
   macro avg     0.7931    0.7820    0.7870      1738
weighted avg     0.8201    0.8228    0.8211      1738

Epoch 3/10
1738/1738 [==============================] - 1s 612us/step
0.7261219792865362
              precision    recall  f1-score   support

         0.0     0.9357   